[View in Colaboratory](https://colab.research.google.com/github/gmihaila/deep_learning_toolbox/blob/master/keras_generator.ipynb)

### Use data Generator in Keras

by GeorgeM.

In [1]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

--2018-05-28 19:23:55--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.01s   

2018-05-28 19:23:55 (1.58 MB/s) - ‘pima-indians-diabetes.data.csv’ saved [23278/23278]



In [14]:
ls

datalab/  pima-indians-diabetes.data.csv


In [22]:
# Create your first MLP in Keras
from keras.models import Sequential
from keras.layers import Dense
import time
import numpy as np

# fix random seed for reproducibility
np.random.seed(7)
# load pima indians dataset
dataset = np.loadtxt('pima-indians-diabetes.data.csv', delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
data_size = len(X)
print 'Data size: ', data_size

data = []
for x,y in zip(X,Y):
  data.append([x,y])
print 'Data created'

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


def gen(data, mini_batch, n_epochs):
    print('Generator initiated!\n')
    # read or split data
    data_size = len(data)
    x = []    # input batch examples
    y = []    # output batch examples
    epoch = 0
    
    while epoch < n_epochs:    # <- VERY IMPORTANT
      for index, pair in enumerate(data):
        # append data        
        x.append(pair[0])
        y.append(pair[1])

        if (index+1)%mini_batch == 0:
          x = np.array(x)
          y = np.array(y)
          # output data
          yield x, y
          x = []; y = []
          
      epoch += 1
    return

Data size:  768
Data created


In [23]:
tr_gen = gen(data, 1, n_epochs=10)
s = time.time()
model.fit_generator(generator=tr_gen, steps_per_epoch=768, epochs=10, max_queue_size=0, workers=0, verbose=2)
e = time.time()
print '\n\n Generator: ',(e-s)


# steps_per_epoch: batches of examples/examples form Generator to finish 1 epoch
# max_queue_size: Integer. Maximum size for the generator queue. Default 10.
# workers: Integer. Maximum number of processes to spin up when using process-based threading. Default to 1. 
         # 0, will execute the generator on the main thread.
# epochs: number of times model sees data



# Fit the model
# s = time.time()
# model.fit(X, Y, epochs=1, batch_size=1)
# e = time.time()
# print '\n\n Generator: ',(e-s)

# evaluate the model

scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

del model

Epoch 1/10
Generator initiated!

 - 2s - loss: 1.7948 - acc: 0.6068
Epoch 2/10
 - 1s - loss: 0.7456 - acc: 0.6302
Epoch 3/10
 - 1s - loss: 0.7028 - acc: 0.6576
Epoch 4/10
 - 1s - loss: 0.6732 - acc: 0.6576
Epoch 5/10
 - 1s - loss: 0.6522 - acc: 0.6836
Epoch 6/10
 - 1s - loss: 0.6369 - acc: 0.6732
Epoch 7/10
 - 1s - loss: 0.6223 - acc: 0.6901
Epoch 8/10
 - 1s - loss: 0.6152 - acc: 0.6823
Epoch 9/10
 - 1s - loss: 0.6047 - acc: 0.7031
Epoch 10/10
 - 1s - loss: 0.5941 - acc: 0.7044


 Generator:  11.5527789593
768/768 [==============================] - 0s 176us/step

acc: 68.10%


In [0]:
# prediciton generator some code

def TestGenerator(path_data, max_queue_size, n_articles):
  x = []    # input batch examples
    
  index = 1
  epoch = 0

  with (open('%s.pickle'%path_data, "rb")) as openfile:
    while True:
      try:
        ID, content, title, media, source, published = pickle.load(openfile)
        x.append(content)

        if ((index)%max_queue_size == 0) or ((n_articles - index) < max_queue_size):
          x = np.array(x)
          # output data
          yield x
          x = []

        index += 1
      except EOFError:
        break
  
test_gen = TestGenerator(path_data='test', max_queue_size=2, n_articles=n_test)  

predict = model.predict_generator(generator=test_gen, steps=5, max_queue_size=10, workers=1)
print predict[0]